In [1]:
from lib.data.DataPreProcessor import DataPreProcessor
from lib.db.repositories.gesamt_repo import GesamtRepository
from lib.config.Configuration import Configuration
from lib.db.connection_factory import ConnectionFactory
from sklearn.model_selection import train_test_split

ConnectionFactory.add_connection(Configuration.db_connection)

In [2]:
pre_processor = DataPreProcessor()
ks_repo = GesamtRepository('Gesamt', Configuration.db_connection)

In [3]:
df = ks_repo.find_all()
df

,index,ID_LK_SK,age_0_34_%,age_35_59_%,age_60+_%,Einw_pro_qm,Landkreis,LK_SK,Bundesland,age_covid_0_34_%,...,KHK,Herzinfarkt,Herzinsuffizienz,Schlaganfall,Diabetes,Asthma,COPD,Krebs,Lebererkrankungen,Immunschwäche
0,0,1001,43.1065,31.1416,25.7519,1577.4400,SK Flensburg,SK,Schleswig-Holstein,35.7143,...,6.3,0.2,3.1,0.3,6.7,7.2,5.5,0.7,5.3,1.6
1,1,1002,43.7705,32.1901,24.0394,2086.3700,SK Kiel,SK,Schleswig-Holstein,32.9749,...,6.5,0.2,3.4,0.3,6.7,5.2,4.8,0.9,5.7,1.6
2,2,1003,36.8364,33.8244,29.3391,1013.9500,SK Lübeck,SK,Schleswig-Holstein,41.5663,...,7.7,0.2,3.6,0.3,8.2,5.8,5.4,1.1,5.7,1.8
3,3,1004,37.3004,33.8709,28.8286,1109.6900,SK Neumünster,SK,Schleswig-Holstein,39.2405,...,6.8,0.2,3.8,0.4,7.5,4.6,5.3,1.0,4.2,1.6
4,4,1051,33.8248,34.2992,31.8760,93.2752,LK Dithmarschen,LK,Schleswig-Holstein,18.3333,...,7.3,0.2,4.3,0.4,7.1,6.3,5.1,1.0,4.3,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,396,16073,27.4926,34.9825,37.5249,102.6570,LK Saalfeld-Rudolstadt,LK,Thüringen,18.3099,...,7.5,0.2,8.7,0.3,9.1,6.6,6.4,1.1,8.0,1.6
397,397,16074,29.5915,36.2488,34.1597,101.8730,LK Saale-Holzland-Kreis,LK,Thüringen,25.0000,...,7.4,0.2,5.9,0.2,8.8,4.9,4.2,1.0,5.5,1.4
398,398,16075,28.8989,35.7138,35.3873,70.2406,LK Saale-Orla-Kreis,LK,Thüringen,16.1074,...,7.2,0.2,6.7,0.3,8.3,4.6,4.1,1.0,6.4,1.4
399,399,16076,26.3583,35.2938,38.3480,116.0300,LK Greiz,LK,Thüringen,15.6800,...,9.7,0.2,7.2,0.4,9.0,4.8,3.8,1.2,9.1,1.3


In [4]:
features_raw, targets_raw = pre_processor.split_features(df.drop([
    'index', 'Landkreis', 'Bundesland', 'ID_LK_SK', 'LK_SK',
    'Prävalenz',
    'Prävalenz_plausibles_Intervall_untere_Grenze',
    'Prävalenz_plausibles_Intervall_obere_Grenze'
], axis=1), ['Sterberate_%'])
features_raw

,age_0_34_%,age_35_59_%,age_60+_%,Einw_pro_qm,age_covid_0_34_%,age_covid_35_59_%,age_covid_60+_%,age_covid_unknown_%,gender_covid_f_%,gender_covid_m_%,...,KHK,Herzinfarkt,Herzinsuffizienz,Schlaganfall,Diabetes,Asthma,COPD,Krebs,Lebererkrankungen,Immunschwäche
0,43.1065,31.1416,25.7519,1577.4400,35.7143,38.0952,26.1905,0.0,48.7805,51.2195,...,6.3,0.2,3.1,0.3,6.7,7.2,5.5,0.7,5.3,1.6
1,43.7705,32.1901,24.0394,2086.3700,32.9749,34.7670,32.2581,0.0,56.8182,43.1818,...,6.5,0.2,3.4,0.3,6.7,5.2,4.8,0.9,5.7,1.6
2,36.8364,33.8244,29.3391,1013.9500,41.5663,39.7590,18.6747,0.0,51.2987,48.7013,...,7.7,0.2,3.6,0.3,8.2,5.8,5.4,1.1,5.7,1.8
3,37.3004,33.8709,28.8286,1109.6900,39.2405,43.0380,17.7215,0.0,44.0000,56.0000,...,6.8,0.2,3.8,0.4,7.5,4.6,5.3,1.0,4.2,1.6
4,33.8248,34.2992,31.8760,93.2752,18.3333,36.6667,45.0000,0.0,52.6316,47.3684,...,7.3,0.2,4.3,0.4,7.1,6.3,5.1,1.0,4.3,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,27.4926,34.9825,37.5249,102.6570,18.3099,46.4789,35.2113,0.0,51.4706,48.5294,...,7.5,0.2,8.7,0.3,9.1,6.6,6.4,1.1,8.0,1.6
397,29.5915,36.2488,34.1597,101.8730,25.0000,43.0556,31.9444,0.0,56.2500,43.7500,...,7.4,0.2,5.9,0.2,8.8,4.9,4.2,1.0,5.5,1.4
398,28.8989,35.7138,35.3873,70.2406,16.1074,36.9128,46.9799,0.0,62.9630,37.0370,...,7.2,0.2,6.7,0.3,8.3,4.6,4.1,1.0,6.4,1.4
399,26.3583,35.2938,38.3480,116.0300,15.6800,38.4000,45.9200,0.0,60.2496,39.7504,...,9.7,0.2,7.2,0.4,9.0,4.8,3.8,1.2,9.1,1.3


In [5]:
features_final = pre_processor.one_hot_encode(features_raw)
features_final = pre_processor.divide_by_hundred(features_final, [
    'age_0_34_%', 'age_35_59_%', 'age_60+_%',
    'age_covid_0_34_%', 'age_covid_35_59_%', 'age_covid_60+_%', 'age_covid_unknown_%',
    'gender_covid_f_%', 'gender_covid_m_%', 'gender_covid_unknown_%',
    'Bluthochdruck', 'KHK', 'Herzinfarkt', 'Herzinsuffizienz', 'Schlaganfall',
    'Diabetes', 'Asthma', 'COPD', 'Krebs', 'Lebererkrankungen', 'Immunschwäche',
])
features_final = pre_processor.log_transform(features_final, ['Einw_pro_qm'])

features_final

,age_0_34_%,age_35_59_%,age_60+_%,Einw_pro_qm,age_covid_0_34_%,age_covid_35_59_%,age_covid_60+_%,age_covid_unknown_%,gender_covid_f_%,gender_covid_m_%,...,KHK,Herzinfarkt,Herzinsuffizienz,Schlaganfall,Diabetes,Asthma,COPD,Krebs,Lebererkrankungen,Immunschwäche
0,0.431065,0.311416,0.257519,7.364192,0.357143,0.380952,0.261905,0.0,0.487805,0.512195,...,0.063,0.002,0.031,0.003,0.067,0.072,0.055,0.007,0.053,0.016
1,0.437705,0.321901,0.240394,7.643660,0.329749,0.347670,0.322581,0.0,0.568182,0.431818,...,0.065,0.002,0.034,0.003,0.067,0.052,0.048,0.009,0.057,0.016
2,0.368364,0.338244,0.293391,6.922595,0.415663,0.397590,0.186747,0.0,0.512987,0.487013,...,0.077,0.002,0.036,0.003,0.082,0.058,0.054,0.011,0.057,0.018
3,0.373004,0.338709,0.288286,7.012737,0.392405,0.430380,0.177215,0.0,0.440000,0.560000,...,0.068,0.002,0.038,0.004,0.075,0.046,0.053,0.010,0.042,0.016
4,0.338248,0.342992,0.318760,4.546218,0.183333,0.366667,0.450000,0.0,0.526316,0.473684,...,0.073,0.002,0.043,0.004,0.071,0.063,0.051,0.010,0.043,0.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,0.274926,0.349825,0.375249,4.641087,0.183099,0.464789,0.352113,0.0,0.514706,0.485294,...,0.075,0.002,0.087,0.003,0.091,0.066,0.064,0.011,0.080,0.016
397,0.295915,0.362488,0.341597,4.633495,0.250000,0.430556,0.319444,0.0,0.562500,0.437500,...,0.074,0.002,0.059,0.002,0.088,0.049,0.042,0.010,0.055,0.014
398,0.288989,0.357138,0.353873,4.266063,0.161074,0.369128,0.469799,0.0,0.629630,0.370370,...,0.072,0.002,0.067,0.003,0.083,0.046,0.041,0.010,0.064,0.014
399,0.263583,0.352938,0.383480,4.762430,0.156800,0.384000,0.459200,0.0,0.602496,0.397504,...,0.097,0.002,0.072,0.004,0.090,0.048,0.038,0.012,0.091,0.013


In [6]:
# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    targets_raw, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 320 samples.
Testing set has 81 samples.


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

linReg = LinearRegression()
linReg.fit(X_train, y_train)
linReg.score(X_test, y_test)

0.45909527039759745

In [8]:
from sklearn.preprocessing import PolynomialFeatures

# Polynomial regression of degree 2
poly = PolynomialFeatures(degree=2)

polyReg2 = LinearRegression()
polyReg2.fit(poly.fit_transform(X_train), y_train)
polyReg2.score(poly.fit_transform(X_test), y_test)

-13.669214454550529

In [9]:
import numpy as np
targets_categoorized = np.round(targets_raw).astype(int)

In [15]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

# 1% intervals
f1 = lambda size, offset : lambda x : str(int(((x + offset) // size) * size)) + '-' + str(int(((x + offset) // size + 1) * size))

targets_categories = pre_processor.one_hot_encode(targets_raw['Sterberate_%'].apply(f1(5, 0)), 'f_')

X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    targets_categories, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

In [16]:
# Build the model architecture
model = Sequential()
model.add(Dense(X_train.shape[1] * 7, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='relu'))
model.summary()

# Compile the model using a loss function and an optimizer.
model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['accuracy'])
#model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 154)               3542      
_________________________________________________________________
dropout_3 (Dropout)          (None, 154)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 620       
Total params: 4,162
Trainable params: 4,162
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, epochs=1000, batch_size=200, verbose=0)

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.8148148059844971


In [21]:
max_interval_size = 10

best_score = 0.0
best_params = (0, 0)
params_score_map = np.zeros((max_interval_size + 1, 1))

for interval_size in range(1, max_interval_size + 1):
    for offset in range (0, 1):
        targets_categories = pre_processor.one_hot_encode(targets_raw['Sterberate_%'].apply(f1(interval_size, offset)), 'f_')
        X_train, X_test, y_train, y_test = train_test_split(features_final, targets_categories, test_size = 0.2, random_state = 0)
        
        # Define the model
        model = Sequential()
        model.add(Dense(X_train.shape[1] * 5, activation='relu', input_dim=X_train.shape[1]))
        model.add(Dropout(0.5))
        model.add(Dense(y_train.shape[1], activation='relu'))

        # Compile the model using a loss function and an optimizer.
        model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=1000, batch_size=200, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=0)
        params_score_map[interval_size, offset] = score[1]
        if (score[1] > best_score):
            best_score = score[1]
            best_params = (interval_size, offset)
        print("Size: " + str(interval_size) + ", Offset: " + str(offset) + ", Accuracy: ", str(score[1]))
        
print("Best score " + str(best_score) + "reached with params " + str(best_params))
params_score_map

Size: 1, Offset: 0, Accuracy:  0.1358024626970291
Size: 2, Offset: 0, Accuracy:  0.4691357910633087
Size: 3, Offset: 0, Accuracy:  0.6419752836227417
Size: 4, Offset: 0, Accuracy:  0.6790123581886292
Size: 5, Offset: 0, Accuracy:  0.8148148059844971
Size: 6, Offset: 0, Accuracy:  0.8518518805503845
Size: 7, Offset: 0, Accuracy:  0.8765432238578796
Size: 8, Offset: 0, Accuracy:  0.9012345671653748
Size: 9, Offset: 0, Accuracy:  0.9382715821266174
Size: 10, Offset: 0, Accuracy:  0.9506173133850098
Best score 0.9506173133850098reached with params (10, 0)


array([[0.        ],
       [0.13580246],
       [0.46913579],
       [0.64197528],
       [0.67901236],
       [0.81481481],
       [0.85185188],
       [0.87654322],
       [0.90123457],
       [0.93827158],
       [0.95061731]])